In [259]:
import pandas as pd
from pathlib import Path
from datetime import datetime
import requests
import zipfile
import shutil
import os

import numpy as np
import matplotlib.pyplot as plt

In [260]:
## filter the warning for now on
#import warnings
#warnings.simplefilter("ignore", DeprecationWarning)

# new way to create an ordered category
from pandas.api.types import CategoricalDtype

---
## Description:

The data set is the full, cleaned results of the 2019 Stack Overflow Developer Survey are in the external subdirectory. Free response submissions and personally identifying information have been removed from the results to protect the privacy of respondents. 

The survey was fielded from January 23 to February 14, 2019. The median time spent on the survey for qualified responses was 23.3 minutes.

Respondents were recruited primarily through channels owned by Stack Overflow. The top 5 sources of respondents were onsite messaging, blog posts, email lists, Meta posts, banner ads, and social media posts. Since respondents were recruited in this way, highly engaged users on Stack Overflow were more likely to notice the links for the survey and click to begin it.

---
## File Locations

There are three files:
1. survey_results_public.csv - CSV file with main survey results, one respondent per row and one column per answer
2. survey_results_schema.csv - CSV file with survey schema, i.e., the questions that correspond to each column name
3. so_survey_2019.pdf - PDF file of survey instrument

A local copy of survey_results_public.csv and survey_results_schema.csv were renamed survey2019.csv and schema2019.csv for further use.

In [261]:
# download the data if its not in the local directory

if not os.path.exists(in_file):
    # put these here in case we want to look at any other years...
    url =  'https://drive.google.com/uc?export=download&id=1QOmVDpd8hcVYqqUXDXf68UMDWQZP0wQV'
    survey_filename = 'survey_results_public.csv'
    questions_filename = 'survey_results_schema.csv'
    year = 2019
    print('Downloading {} survey'.format(year))

    request = requests.get(url)
    with open('survey.zip', 'wb') as file:
        file.write(request.content) 

    print('Unzipping {} survey'.format(year))
    with zipfile.ZipFile("survey.zip", "r") as file:
        file.extractall("data/external/")

    print('Moving {} survey'.format(year))
    shutil.copytree('data/external', 'data/raw')
    shutil.copy('data/external/' + survey_filename, 'data/raw/survey{}.csv'.format(year))
    shutil.copy('data/external/' + questions_filename, 'data/raw/schema{}.csv'.format(year))
    
    print('cleaning up')
    os.remove('survey.zip')

In [262]:
today = datetime.today()
in_file = Path.cwd() / "data" / "raw" / "survey2019.csv"
# this is where our cleaned up file will put put
summary_file = Path.cwd() / "data" / "processed" / f"summary_{today:%b-%d-%Y}.pkl"

---
## Prepare the data

- Remove all leading and trailing spaces (not nescessary)
- Rename the columns for consistency (not nescessary)
- Convert objects to category

In [263]:
#beware that the Respondent ids start at 1
df = pd.read_csv(in_file, index_col='Respondent')

In [268]:
# Print column name
df.columns 

Index(['MainBranch', 'Hobbyist', 'OpenSourcer', 'OpenSource', 'Employment',
       'Country', 'Student', 'EdLevel', 'UndergradMajor', 'EduOther',
       'OrgSize', 'DevType', 'YearsCode', 'Age1stCode', 'YearsCodePro',
       'CareerSat', 'JobSat', 'MgrIdiot', 'MgrMoney', 'MgrWant', 'JobSeek',
       'LastHireDate', 'LastInt', 'FizzBuzz', 'JobFactors', 'ResumeUpdate',
       'CurrencySymbol', 'CurrencyDesc', 'CompTotal', 'CompFreq',
       'ConvertedComp', 'WorkWeekHrs', 'WorkPlan', 'WorkChallenge',
       'WorkRemote', 'WorkLoc', 'ImpSyn', 'CodeRev', 'CodeRevHrs', 'UnitTests',
       'PurchaseHow', 'PurchaseWhat', 'LanguageWorkedWith',
       'LanguageDesireNextYear', 'DatabaseWorkedWith',
       'DatabaseDesireNextYear', 'PlatformWorkedWith',
       'PlatformDesireNextYear', 'WebFrameWorkedWith',
       'WebFrameDesireNextYear', 'MiscTechWorkedWith',
       'MiscTechDesireNextYear', 'DevEnviron', 'OpSys', 'Containers',
       'BlockchainOrg', 'BlockchainIs', 'BetterLife', 'ITperson', 

### zero order sanity checks

- column names
- complete columns (no `nan`s)
- look for highly unpopulated columns (nan > 50%)

Check for empty entries...  complete columns... and columns with lots of missing data 

In [271]:
# what pct of of empty entries..
(100 * df.isna().sum() / df.shape[0]).describe()

count    84.000000
mean     13.780154
std      12.171536
min       0.000000
25%       2.128078
50%      10.792277
75%      22.270007
max      45.799534
dtype: float64

not too many holes in the data.

How many complete columns?

In [274]:
# check columns with no missing values
set(df.isna().sum()[df.isna().sum()==0].keys())

{'Hobbyist', 'OpenSourcer'}

In [275]:
# check which columns have over 50% missing values
sum(df.columns[100*df.isna().sum()/df.shape[0] > 50])

0

---

## Create derived data columns

- simplifications
- summaries
- multi-select splits


### convert some columns to categorical

- currently 'objects' containing strings.

In [286]:
# only convert categories we want to convert... 
convert_to_cat = ['MainBranch', 'Hobbyist', 'OpenSourcer',  'Employment',
       'Country', 'Student', 'UndergradMajor',
       'OrgSize', 'DevType',
       'CareerSat',  'MgrIdiot', 'MgrMoney', 'MgrWant', 'FizzBuzz',
       'CurrencySymbol', 'CurrencyDesc', 'CompTotal', 'CompFreq', 'WorkWeekHrs', 'WorkPlan',
       'WorkRemote', 'WorkLoc',  'CodeRev', 'CodeRevHrs', 'UnitTests',
       'OpSys', 'BlockchainOrg', 'BlockchainIs', 'BetterLife', 'ITperson', 'OffOn',
       'SocialMedia', 'Extraversion','Age', 'Gender', 'Trans',
       'Sexuality', 'Ethnicity', 'Dependents', 'SurveyLength', 'SurveyEase','ScreenName']

# convert to categories
for col_name in convert_to_cat:
    if(df[col_name].dtype == 'object'):
        df[col_name] = df[col_name].astype('category')
        #df[col_name] = df[col_name].cat.codes
        
# convert to ordered categories below
#     'EdLevel','OpenSource','JobSat','ImpSyn',

### convert columns to ordered categories

- eductaion level
- openSource, career/job satisfaction
- personal competence (ImpSyn)

In [287]:
# ordered categories 
# Education
edlevel_cat = CategoricalDtype( ['I never completed any formal education',
                                 'Primary/elementary school',
                                 'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
                                 'Some college/university study without earning a degree',
                                 'Associate degree',
                                 'Bachelor’s degree (BA, BS, B.Eng., etc.)',
                                 'Master’s degree (MA, MS, M.Eng., MBA, etc.)',
                                 'Professional degree (JD, MD, etc.)',
                                 'Other doctoral degree (Ph.D, Ed.D., etc.)'],ordered=True)

df['EdLevel'] = df.EdLevel.astype(edlevel_cat)

#OpenSource . Higher, same lower
opensource_cat = CategoricalDtype( ['OSS is, on average, of HIGHER quality than proprietary / closed source software',
                                    'The quality of OSS and closed source software is about the same',
                                    'OSS is, on average, of LOWER quality than proprietary / closed source software',], ordered=True)
df['OpenSource'] = df.OpenSource.astype(opensource_cat)
 
# career satisfaction / job satisfaction\
careersat_cat = CategoricalDtype(['Very dissatisfied',
                                     'Slightly dissatisfied',
                                     'Neither satisfied nor dissatisfied',
                                     'Slightly satisfied',
                                     'Very satisfied'], ordered=True)
df['CareerSat'] = df.CareerSat.astype(careersat_cat)

jobsat_cat = CategoricalDtype(['Very dissatisfied',
                                     'Slightly dissatisfied',
                                     'Neither satisfied nor dissatisfied',
                                     'Slightly satisfied',
                                     'Very satisfied'], ordered=True)
df['JobSat'] = df.JobSat.astype(jobsat_cat)

#competence
impsyn_cat = CategoricalDtype( ['Far below average',
                                 'A little below average',
                                 'Average',
                                 'A little above average',
                                 'Far above average'], ordered=True)
df['ImpSyn']  = df.ImpSyn.astype(impsyn_cat)


In [288]:
print('The dataset contains', df.shape[0], 'rows and', df.shape[1], 'columns.')

The dataset contains 88883 rows and 86 columns.


### Create derived data columns

- Simplify some categories
    - Education: Advanced, College, Grade
    - Major: Computer/Tech, Math/Science, Other
        
- generation (from Age)
   - Age->Gen: GenZ,Milennial,GenX,Boomer,Silent
           

In [289]:
# Print shape of dataset
#print('The dataset contains', np.shape(df)[0], 'rows and', np.shape(df)[1], 'columns.')
print('The dataset contains', df.shape[0], 'rows and', df.shape[1], 'columns.')


The dataset contains 88883 rows and 86 columns.


In [290]:
df['EdLevel'].cat.categories

Index(['I never completed any formal education', 'Primary/elementary school',
       'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
       'Some college/university study without earning a degree',
       'Associate degree', 'Bachelor’s degree (BA, BS, B.Eng., etc.)',
       'Master’s degree (MA, MS, M.Eng., MBA, etc.)',
       'Professional degree (JD, MD, etc.)',
       'Other doctoral degree (Ph.D, Ed.D., etc.)'],
      dtype='object')

In [280]:
# old method comparing strings

advanced = df['EdLevel'].isin(['Other doctoral degree (Ph.D, Ed.D., etc.)',
                               'Master’s degree (MA, MS, M.Eng., MBA, etc.)',
                               'Professional degree (JD, MD, etc.)'])

college = df['EdLevel'].isin(['Associate degree',
                              'Bachelor’s degree (BA, BS, B.Eng., etc.)',
                              'Some college/university study without earning a degree'])

grade = df['EdLevel'].isin(['Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
                            'Primary/elementary school',
                            'I never completed any formal education'])
no_answer = df['EdLevel'].isnull()

print('The dataset contains',  advanced.sum(),
      'respondents with advanced degrees, (n=', college.sum(),
      'college; n=', grade.sum(), 'grade) ', (advanced.sum() +
                                              college.sum()+grade.sum()+no_answer.sum()),
      'og~', df.shape[0],
      'null->', no_answer.sum())

The dataset contains 23199 respondents with advanced degrees, (n= 52574 college; n= 10617 grade)  88883 og~ 88883 null-> 2493


In [281]:
# create ordered category
edlevel_simple_cat = CategoricalDtype(  ['Grade',
                                 'College', #'N/A'
                                 'Advanced'], ordered=True)
df.loc[:,'Education'] = np.nan

df.loc[advanced,'Education'] = 'Advanced'
df.loc[college,'Education'] = 'College'
df.loc[grade,'Education'] = 'Grade'
#df.loc[no_answer,'Education'] = 'N/A'
#df.loc[:,'Education'] = np.nan
# now make it an ordered category
df.loc[:,'Education']  = df.Education.astype(edlevel_simple_cat)



In [282]:
# #method using ordered category (is awkward because of nan)
# if False:
#     advanced = (survey['EdLevel'].dropna() >= survey.EdLevel.cat.categories[6])

#     college = ((survey['EdLevel'].dropna() < survey.EdLevel.cat.categories[6]) & (survey['EdLevel'].dropna() >= survey.EdLevel.cat.categories[3])) 
  
#     grade = (survey['EdLevel'].dropna() < survey.EdLevel.cat.categories[3])
#     no_answer = survey['EdLevel'].isnull()

    
#     print('The dataset contains',  advanced.sum(),  
#           'respondents with advanced degrees, (n=', college.sum(), 
#           'college; n=', grade.sum(), 'grade) ', (advanced.sum()+college.sum()+grade.sum()+no_answer.sum() ),
#          'og~',survey.shape[0], 
#          'null->', no_answer.sum())


In [283]:
df['Education'].head(3)

Respondent
1      Grade
2      Grade
3    College
Name: Education, dtype: category
Categories (3, object): [Grade < College < Advanced]

In [284]:
df.loc[:,'Major'] = np.nan   

computer_tech = df['UndergradMajor'].isin(['Computer science, computer engineering, or software engineering',
                                        'Web development or web design',
                                        'Information systems, information technology, or system administration'])
   

math_sci = df['UndergradMajor'].isin(['Mathematics or statistics',
                                          'Another engineering discipline (ex. civil, electrical, mechanical)',
                                          'A natural science (ex. biology, chemistry, physics)'])
       
other = df['UndergradMajor'].isin(['A health science (ex. nursing, pharmacy, radiology)',
                                       'A business discipline (ex. accounting, finance, marketing)',
                                       'A humanities discipline (ex. literature, history, philosophy)',
                                       'A social science (ex. anthropology, psychology, political science)',
                                       'Fine arts or performing arts (ex. graphic design, music, studio art)'])
 

df.loc[computer_tech,'Major'] = 'Computer/Tech'
df.loc[math_sci,'Major'] = 'Math/Science'
df.loc[other,'Major'] = 'Other'
#df.loc[no_answer,'EdLevel_simple'] = 'N/A'

df.loc[:,'Major']  = df.Major.astype('category')

In [285]:
df['Major'].head()
#survey.UndergradMajor.cat.categories

Respondent
1              NaN
2              NaN
3    Computer/Tech
4    Computer/Tech
5    Computer/Tech
Name: Major, dtype: category
Categories (3, object): [Computer/Tech, Math/Science, Other]

### fix some nonsensical numbers

---
Age: `Age`-> `nAge`  change < 13, or > 80 to `NaN`
1st Code: `Age1stCode` -> `nAgeCode` , change "Younger than 5 years" to 4.5 and "Older than 85" to 85.
years coding, years experience: `YearsCode` -> `nYearsCode`, change "Less than 1 year" to 1/10000 and "More than 50" to 55.
years pro:`YearsCodePro` -> `nYearsPro`
workweek: `WorkWeekHrs` to require 4 hours off per day

---
Test that `ConvertedComp` is sensible?

In [24]:
def fix_years_coding(years_):
    # make these numeric in a sensible way
    if years_ == 'Less than 1 year':
        return 0.0001
    elif years_ == 'More than 50 years':
        return 55.
    else:
        return years_

    
def fix_age(years_):
    # make these numeric in a sensible way  exclude pre-teens and aged >84
    if years_ < 13:
        return np.NaN
    elif years_ > 84:
        return np.NaN
    else:
        return years_ 
    
def fix_age1stcode(years_):
    if years_ == 'Younger than 5 years':
        return 4.5
    elif years_ == 'Older than 85':
        return 85.
    else:
        return years_


def fix_workweekhours(hours_):
    #if hours_ == np.NaN:
    #    return hours_
    if hours_ > 20*7:  # hard ceiling on hours in a week (24*7)  lets throw out the ridiculous no sleep 24/7 scenarios
        return np.NaN
    elif hours_ < 4: # arbitrary minimum work time
        return np.NaN
    else:
        return hours_

    

In [25]:
df.loc[:,'nYearsCode'] =  pd.to_numeric(df.YearsCode.apply(fix_years_coding))
df.loc[:,'nYearsPro'] = pd.to_numeric(df.YearsCodePro.apply(fix_years_coding))
df.loc[:,'WorkWeekH'] = df.WorkWeekHrs.apply(fix_workweekhours)
df.loc[:,'nAge'] =  pd.to_numeric(df.Age.apply(fix_age))
df.loc[:,'nAgeCode'] =  pd.to_numeric(df.Age1stCode.apply(fix_age1stcode))


In [26]:
df.Age.describe()

count    79210.000000
mean        30.336699
std          9.178390
min          1.000000
25%         24.000000
50%         29.000000
75%         35.000000
max         99.000000
Name: Age, dtype: float64

In [27]:
df['WorkWeekH'].dropna().describe()
# looks good

count    64017.000000
mean        40.376339
std         10.092518
min          4.000000
25%         40.000000
50%         40.000000
75%         44.000000
max        140.000000
Name: WorkWeekH, dtype: float64

In [28]:
df.groupby('Dependents').ConvertedComp.agg(['count','mean','median']).sort_values('mean')


,count,mean,median
Dependents,,,
Yes,21803,127388.806816,61872.0
No,32559,127992.870788,54996.0


###  Categorize according to standard "generations"

- Gen Z
- Millenial
- Gen X
- Boomers
- Silent

![alt text][logo]

[logo]:
https://www.pewresearch.org/wp-content/uploads/2019/01/FT_19.01.17_generations_2019.png


In [29]:
# Convert Age to generation with 2019 as current time reference
def find_gen(age):
    """
    """
    if age <= 22:
        gen = 'GenZ'
        return gen
    elif age <= 38:
        gen = 'Millenial'
    elif age <= 54:
        gen = 'GenX'
    elif age <= 73:
        gen = 'Boomer'
    else:
        gen = 'Silent'
    return gen


generation_cat = CategoricalDtype(['GenZ',
                                   'Millenial',
                                   'GenX',
                                   'Boomer',
                                   'Silent',  # 'N/A'
                                   ], ordered=True)

In [30]:
# Apply function to subsets
df.loc[:,'Gen'] = np.nan
df.loc[:,'Gen'] = df.Age.apply(find_gen).copy()
df.loc[:,'Gen']  = df.Gen.astype(generation_cat)
df.groupby('Gen').ConvertedComp.agg(['count','mean','median']).sort_values('mean')


,count,mean,median
Gen,,,
GenZ,3874,67796.219411,19266.0
Millenial,40895,121602.788654,53500.0
Silent,2324,138549.665232,60000.0
GenX,7756,172946.333935,91660.0
Boomer,974,202005.099589,110000.0


In [31]:
# these will need to be "split" later
treat_as_lists = ['JobFactors','DevEnviron',
                 'Containers','WorkChallenge',
                 'LanguageWorkedWith', 'LanguageDesireNextYear', 
                 'DatabaseWorkedWith','DatabaseDesireNextYear', 
                 'MiscTechWorkedWith','MiscTechDesireNextYear', 
                 'EduOther']
df.dtypes[df.dtypes==('object')]

EduOther                  object
YearsCode                 object
Age1stCode                object
YearsCodePro              object
JobSeek                   object
LastHireDate              object
LastInt                   object
JobFactors                object
ResumeUpdate              object
WorkChallenge             object
PurchaseHow               object
PurchaseWhat              object
LanguageWorkedWith        object
LanguageDesireNextYear    object
DatabaseWorkedWith        object
DatabaseDesireNextYear    object
PlatformWorkedWith        object
PlatformDesireNextYear    object
WebFrameWorkedWith        object
WebFrameDesireNextYear    object
MiscTechWorkedWith        object
MiscTechDesireNextYear    object
DevEnviron                object
Containers                object
SOVisit1st                object
SOVisitFreq               object
SOVisitTo                 object
SOFindAnswer              object
SOTimeSaved               object
SOHowMuchTime             object
SOAccount 

----
## Clean Data

### Respondent Cleanup

- KEEP PROFESSIONALS ONLY.  Dropped students and "other"
- Create our categories of respondents (aux columns)
    - DataScientist
    - Non-DataScientists (everybody else, including un-engaged, jokers, and students)
    - Developers (other "professionals")


In [32]:
df['MainBranch'].unique()

[I am a student who is learning to code, I am not primarily a developer, but I write co..., I am a developer by profession, I code primarily as a hobby, I used to be a developer by profession, but no..., NaN]
Categories (5, object): [I am a student who is learning to code, I am not primarily a developer, but I write co..., I am a developer by profession, I code primarily as a hobby, I used to be a developer by profession, but no...]

As we are interested in comparing data scientists to non-data scientists, we need to be able to differentiate between the two. This is done using the `DevType` field. As a result, we should drop any rows where this field is missing, since we can't determine which subset these rows fit into.


In [33]:
#make some of the categories ordered
developers = df['MainBranch'] == 'I am a developer by profession' # df['MainBranch'].cat.categories[0]
# limit our scope to professional developers
df = df[developers]

In [34]:
print('The dataset contains', df.shape[0], 'rows and', df.shape[1], 'columns.')

The dataset contains 65679 rows and 92 columns.


In [35]:
# Create data scientist and non-data scientist subsets.
# data scientists are defined as "Data or business analyst",Engineer, Data", "Data scientist or machine learning specialist"
# Database administrator or Scientist

# developer is "Developer, {fullstack, game or graphics, mobile, QA or test}"
data_scientist = df['DevType'].str.contains('|'.join(['data','scientist']), case=False, na=False, regex=True).copy() #data scientists / analists / Data engineers

#data = df['DevType'].str.contains('data', case=False, na=False, regex=True).copy() #data scientists / analists / Data engineers
developer = df['DevType'].str.contains('developer', case=False, na=False,regex=True).copy() #all types of developers

In [36]:
# its way too messy to parse through the multi-selects... 
df.DevType[data_scientist].unique()
#df.DevType[developer].unique()


[Database administrator;Developer, back-end;Dev..., Data or business analyst;Data scientist or mac..., Data or business analyst;Database administrato..., Data or business analyst;Designer;Developer, b..., Academic researcher;Data scientist or machine ..., ..., Developer, back-end;Developer, desktop or ente..., Data scientist or machine learning specialist;..., Data scientist or machine learning specialist;..., Data scientist or machine learning specialist;..., Academic researcher;Database administrator;Dev...]
Length: 6805
Categories (6805, object): [Database administrator;Developer, back-end;Dev..., Data or business analyst;Data scientist or mac..., Data or business analyst;Database administrato..., Data or business analyst;Designer;Developer, b..., ..., Data scientist or machine learning specialist;..., Data scientist or machine learning specialist;..., Data scientist or machine learning specialist;..., Academic researcher;Database administrator;Dev...]

In [37]:
data_scientist.sum()

15151

In [41]:
# make some expository columns
df.loc[:,'DSorDV'] = np.nan

df.loc[data_scientist,'DSorDV'] = 'Data Scientist'
df.loc[developer,'DSorDV'] = 'Developer'

df.loc[:,'isDS'] = data_scientist
df.loc[:,'isDev'] = developer
df.loc[:,'notDS'] = ~data_scientist

### Column Cleanup
- Drop columns which we don't want to anlayze
- re-order/group columns (in case we want to select with slices?)
- Remove all leading and trailing spaces (not nescessary)
- Rename the columns for consistency (not nescessary)

In [44]:
df.columns

Index(['MainBranch', 'Hobbyist', 'OpenSourcer', 'OpenSource', 'Employment',
       'Country', 'Student', 'EdLevel', 'UndergradMajor', 'EduOther',
       'OrgSize', 'DevType', 'YearsCode', 'Age1stCode', 'YearsCodePro',
       'CareerSat', 'JobSat', 'MgrIdiot', 'MgrMoney', 'MgrWant', 'JobSeek',
       'LastHireDate', 'LastInt', 'FizzBuzz', 'JobFactors', 'ResumeUpdate',
       'CurrencySymbol', 'CurrencyDesc', 'CompTotal', 'CompFreq',
       'ConvertedComp', 'WorkWeekHrs', 'WorkPlan', 'WorkChallenge',
       'WorkRemote', 'WorkLoc', 'ImpSyn', 'CodeRev', 'CodeRevHrs', 'UnitTests',
       'PurchaseHow', 'PurchaseWhat', 'LanguageWorkedWith',
       'LanguageDesireNextYear', 'DatabaseWorkedWith',
       'DatabaseDesireNextYear', 'PlatformWorkedWith',
       'PlatformDesireNextYear', 'WebFrameWorkedWith',
       'WebFrameDesireNextYear', 'MiscTechWorkedWith',
       'MiscTechDesireNextYear', 'DevEnviron', 'OpSys', 'Containers',
       'BlockchainOrg', 'BlockchainIs', 'BetterLife', 'ITperson', 

In [45]:
new_columns = ['Education', 'Major', 'nYearsCode','nAgeCode','nAge'
               'nYearsPro', 'WorkWeekH', 'Gen', 'isDS', 'isDev', 'notDS']

In [46]:
# Drop unnecessary columns
keep_columns = [
        'MainBranch',  # should be only Developers so could drop
        #
        'Employment',
        'Country',
        'Student',
        #
        'EdLevel',
        'UndergradMajor',
        'Major',
        'Education',
        #
        'OrgSize',
        'DevType',
        'isDS', 'isDev', 'notDS', 'DSorDV',
        # Years  -treat as numbers
        'nYearsCode',  # 'YearsCode',
        'nYearsPro',  # 'YearsCodePro',
        'nAgeCode',  # 'Age1stCide'
        'WorkWeekH',  # 'WorkWeekHrs'
        #
        'ImpSyn',  # PRIMARY  # Self - competence !!!
        'CareerSat',
        'JobSat',
        'MgrIdiot',  # MGR competence
        'MgrMoney',  # do you need to be a manager to make $$
        'MgrWant',  # do you want to be a manager
        # 'JobFactors', # what drives values - grouped with other list types below
        'ConvertedComp',  # PRIMARY
        #
        'OpenSourcer',
        'OpenSource',
        #
        'OpSys',
        'BlockchainOrg', 'BlockchainIs',
        # personality
        'BetterLife',
        'ITperson', 'OffOn',
        #  social / demographic
        'SocialMedia',
        'Extraversion',
        'ScreenName',
        'nAge', 'Gen',
        'Gender', 'Trans',
        'Sexuality', 'Ethnicity', 'Dependents',
        # do these pairs  helps is understand how they see the future? (maybe drop)
        'JobFactors', 'DevEnviron',
        'Containers', 'WorkChallenge',
        'LanguageWorkedWith', 'LanguageDesireNextYear',
        'DatabaseWorkedWith', 'DatabaseDesireNextYear',
        'MiscTechWorkedWith', 'MiscTechDesireNextYear',
        'PlatformWorkedWith', 'PlatformDesireNextYear',
        'EduOther',
        # might want to look at these later.
        'Hobbyist',
        'FizzBuzz',
        'ResumeUpdate',
        'CurrencySymbol', 'CurrencyDesc',
        'CompTotal', 'CompFreq',
        'SurveyLength', 'SurveyEase']


df = df[keep_columns]

In [47]:
# SKIP ALL STACKOVERFLOW PARTICULARS
#'JobSeek','LastHireDate', 'LastInt',
#'WorkPlan', 'WorkChallenge',
#' WorkRemote', 'WorkLoc',
#'CodeRev', 'CodeRevHrs', 'UnitTests',
#'PurchaseHow', 'PurchaseWhat',
# Skip these
# 'WebFrameWorkedWith','WebFrameDesireNextYear',
# Print shape of dataset
print('The dataset now contains',
      df.shape[0], 'rows and', df.shape[1], 'columns.')

The dataset now contains 65679 rows and 65 columns.


----
## Finalized Cleaning

In [48]:
#cols_to_rename = {'col1': 'New_Name'}
#df.rename(columns=cols_to_rename, inplace=True)

### Clean Up Data Types

### Create some categories if nescessary

In [49]:
#df.dtypes

In [50]:
#df?

---
## Save Preprocessed Data

### Save output file into processed directory

Save a file in the processed directory that is cleaned properly. It will be read in and used later for further analysis.

Other options besides pickle include:
- feather
- msgpack
- parquet

In [51]:
df.to_pickle(summary_file)

summary_file

PosixPath('/Users/ergonyc/Projects/Insight/Modules/DataPresentation0/data/processed/summary_Sep-04-2019.pkl')

### Create categories of respondents t



---
## Split Multi-Selection Fields


For the fields where multiple selections were possible (e.g.  `JobFactors`,`DevEnviron`, and `LanguageWorkedWith`), split the strings containing the multiple selections into a list of selections and then concatenate these lists into a single list (dropping any missing values in the process). 

We may also want to simplify these fields to reduce the length of category labels and to group similar categories into a single category apon further analysis.

Full list of the multi-select variables we will be addressing:
 `JobFactors`,`DevEnviron`,`Containers`,`WorkChallenge`, as well as the current/future pairs:
     `LanguageWorkedWith`, `LanguageDesireNextYear`, 
     `DatabaseWorkedWith`,`DatabaseDesireNextYear`, 
     `MiscTechWorkedWith`,`MiscTechDesireNextYear`, 


### Languages 

`LanguageWorkedWith`, `LanguageDesireNextYear`, 

Should I convert these to categories?  maybe I should just prepare then, but not break into boolean fields until analysis?

In [52]:
#languages_full = survey['LanguageWorkedWith'].dropna().str.split(';').tolist()

In [53]:
# We have to split the LanguageWorkedWith to get a proper picture
#languages = survey[survey['LanguageWorkedWith'].notnull()]

col = 'LanguageWorkedWith' 

unique_selects = {}

select_na = df[col].isnull()
# split the languages on ;
#for language_set in survey['LanguageWorkedWith'].dropna().apply(lambda row: str(row).split(';')) :
for select_set in df[col].apply(lambda row: str(row).split(';')) :
    for select in select_set:
        if select not in unique_selects.keys():
            unique_selects[select] = 1
        else:
            unique_selects[select] += 1
            
un_sel = pd.Series(unique_selects).sort_values(ascending=False).copy()
#un_sel.index == 'nan'
un_sel

JavaScript               46945
HTML/CSS                 41911
SQL                      37235
Java                     26348
Python                   24285
Bash/Shell/PowerShell    24195
C#                       21657
PHP                      17042
TypeScript               16535
C++                      13296
C                        11130
Ruby                      5923
Go                        5852
Other(s):                 5596
Swift                     4652
Kotlin                    4590
Objective-C               3615
Assembly                  3173
VBA                       2987
Scala                     2805
R                         2731
Rust                      1981
Dart                      1244
Elixir                    1064
Clojure                   1029
F#                         757
WebAssembly                745
Erlang                     622
nan                        405
dtype: int64

In [106]:
out_df = pd.DataFrame() #columns = unique_selects)

prefix = col[:20]
un_sel.index

Index(['JavaScript', 'HTML/CSS', 'SQL', 'Java', 'Python',
       'Bash/Shell/PowerShell', 'C#', 'PHP', 'TypeScript', 'C++', 'C', 'Ruby',
       'Go', 'Other(s):', 'Swift', 'Kotlin', 'Objective-C', 'Assembly', 'VBA',
       'Scala', 'R', 'Rust', 'Dart', 'Elixir', 'Clojure', 'F#', 'WebAssembly',
       'Erlang', 'nan'],
      dtype='object')

In [107]:
for sel in un_sel.index:
    col_name = col + sel.replace(' ','_')
    if (sel == 'nan'):
        out_df.loc[:,col_name] = select_na

    else:
        out_df.loc[:,col_name] = ~select_na
        selected = df[col].dropna().str.split(';').copy() #.tolist()
        # need to strip the nulls
        out_df.loc[~select_na,col_name] = selected.apply(lambda x: sel in x)
        out_df.loc[select_na,col_name] = np.nan

        
print(select_na.shape)
print(out_df.shape)


(65679,)
(65679, 29)


In [110]:
(100*out_df.sum().sort_values(ascending=False)/out_df.shape[0])

numLang = out_df.sum(axis =1)


5.029765983038718

In [151]:
# We have to split the LanguageWorkedWith to get a proper picture
#languages = survey[survey['LanguageWorkedWith'].notnull()] 
def split_multiselect(df, col):
    """Create a new dataframe that splits the values of multi-selection column col into individual selections and 
    places each selection into a boolean column. This new dataframe can be merged into the original dataframe by 
    Respondent (index)value.
    
    Args:
    df: dataframe. Dataframe containing the multi-selection field col.
    col:  column name
    
    Returns:
    out_df: dataframe. New dataframe giving split values of col.
        """

    unique_selects = {}

    select_na = df[col].isnull()
    # split the languages on ;
    #for language_set in survey['LanguageWorkedWith'].dropna().apply(lambda row: str(row).split(';')) :
    for select_set in df[col].apply(lambda row: str(row).split(';')) :
        for select in select_set:
            if select not in unique_selects.keys():
                unique_selects[select] = 1
            else:
                unique_selects[select] += 1

    un_sel = pd.Series(unique_selects).sort_values(ascending=False).copy()
    #un_sel.index == 'nan'
    out_df = pd.DataFrame() #columns = unique_selects)
    new_cols = []
    for sel in un_sel.index:
        col_name = col[:25] + '_' + sel.replace(' ','_')
        new_cols.append(col_name)
        if (sel == 'nan'):
            out_df.loc[:,col_name] = select_na
        else:
            out_df.loc[:,col_name] = ~ select_na #df[col]
            selected = df[col].dropna().str.split(';').copy() #.tolist()
            # need to strip the nulls
            out_df.loc[~select_na,col_name] = selected.apply(lambda x: sel in x)
            out_df.loc[select_na,col_name] = np.nan

    return out_df, new_cols


In [167]:
langs, lang_cols = split_multiselect(df, 'LanguageWorkedWith')
langsfuture, langfuture_cols = split_multiselect(df, 'LanguageDesireNextYear')


In [168]:
df_ = df.merge(langs, how='outer', left_index=True, right_index=True)
df_.shape


(65679, 94)

In [169]:
# how many languages do each respondent claim :  5!!!
df_.loc[:,lang_cols].sum(axis = 1).mean()

5.029765983038718

In [161]:
# percent of developers languages... check NaN...should be nonzero 405/65679
(100*df_.loc[:,lang_cols].sum().sort_values(ascending=False)/df_.shape[0])

LanguageWorkedWith_JavaScript               71.476423
LanguageWorkedWith_HTML/CSS                 63.811873
LanguageWorkedWith_SQL                      56.692398
LanguageWorkedWith_Java                     40.116323
LanguageWorkedWith_Python                   36.975289
LanguageWorkedWith_Bash/Shell/PowerShell    36.838259
LanguageWorkedWith_C#                       32.974010
LanguageWorkedWith_PHP                      25.947411
LanguageWorkedWith_TypeScript               25.175475
LanguageWorkedWith_C++                      20.243914
LanguageWorkedWith_C                        16.946056
LanguageWorkedWith_Ruby                      9.018103
LanguageWorkedWith_Go                        8.910002
LanguageWorkedWith_Other(s):                 8.520227
LanguageWorkedWith_Swift                     7.082934
LanguageWorkedWith_Kotlin                    6.988535
LanguageWorkedWith_Objective-C               5.504042
LanguageWorkedWith_Assembly                  4.831072
LanguageWorkedWith_VBA      

### Development Environment 

`DevEnviron`

In [173]:
devenv, devenv_cols = split_multiselect(df, 'DevEnviron')


In [174]:
# how many environs do each respondent claim :  almost 3!!!
devenv.sum(axis = 1).describe()


count    65679.000000
mean         2.826946
std          1.599255
min          1.000000
25%          2.000000
50%          3.000000
75%          4.000000
max         22.000000
dtype: float64

### JobFactor & WorkChallenges 

`JobFactors`,`WorkChallenge` 

In [180]:

jobfact, jobfact_cols = split_multiselect(df, 'JobFactors')
jobchal, jobchal_cols = split_multiselect(df, 'WorkChallenge')



### MISC - containers, dababases, and misc tech

`Containers`
      `DatabaseWorkedWith`,`DatabaseDesireNextYear`, 
     `MiscTechWorkedWith`,`MiscTechDesireNextYear`, 

In [176]:
containers, containers_cols = split_multiselect(df, 'Containers')
database, database_cols = split_multiselect(df, 'DatabaseWorkedWith')

In [177]:
misctech, misctech_cols = split_multiselect(df, 'MiscTechWorkedWith')

In [258]:
jobchal.sum(axis=1)
jobchal.WorkChallenge_nan.sum()
jobchal[0:end-1].sum()

Respondent
4        1.0
5        3.0
7        3.0
9        3.0
10       1.0
13       3.0
14       2.0
16       3.0
17       3.0
19       3.0
22       3.0
23       3.0
24       3.0
25       1.0
26       3.0
28       1.0
29       3.0
30       2.0
32       2.0
33       3.0
35       3.0
36       2.0
38       3.0
39       3.0
41       3.0
42       3.0
43       3.0
44       3.0
46       3.0
47       3.0
        ... 
88844    2.0
88845    2.0
88846    3.0
88847    3.0
88848    3.0
88850    2.0
88852    3.0
88853    3.0
88854    3.0
88855    3.0
88856    1.0
88857    3.0
88858    3.0
88860    3.0
88865    3.0
88866    3.0
88867    3.0
88868    3.0
88869    3.0
88871    3.0
88872    3.0
88873    1.0
88874    2.0
88876    3.0
88877    2.0
88878    2.0
88879    3.0
88881    2.0
88882    3.0
88883    3.0
Length: 65679, dtype: float64

In [ ]:
df.pivot_table(['Total day calls', 'Total eve calls', 'Total night calls'],
               ['Area code'], aggfunc='mean')

pd.crosstab(df['Churn'], df['Voice mail plan'], normalize=True)


# Plotting a bar graph of the number of stores in each city, for the first ten cities listed
# in the column 'City'
city_count  = df['City'].value_counts()
city_count = city_count[:10,]
plt.figure(figsize=(10,5))
sns.barplot(city_count.index, city_count.values, alpha=0.8)
plt.title('Starbucks in top 10 cities in the World')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('city', fontsize=12)
plt.show()

---

dont 

---

go 

---

below

---

here

---

---
## CODE SNIPPITS for further analysis


### advice from Matt

In [ ]:
if False:
    # these are some notes from matt on now to figure some things out...

    df.loc[df['column'].isin(['value_1', 'value_2']), 'other_column']

    if (value == value):  # Won't return true for nans (edited) 
        x = np.nan


# Look at descriptive statistics for data (ignore Respondent since this is just an ID field)
#survey.drop(['Respondent'], axis = 1).describe()

### Dummies

#METHOD 1
# create the 'Sex_male' dummy variable using the 'map' method
train['Sex_male'] = train.Sex.map({'female':0, 'male':1})
train.head()

# alternative: use 'get_dummies' to create one column for every possible value
pd.get_dummies(train.Sex).head()

# METHOD 2
# drop the first dummy variable ('female') using the 'iloc' method
pd.get_dummies(train.Sex).iloc[:, 1:].head()

# drop the first dummy variable ('female') using the 'iloc' method
pd.get_dummies(train.Sex).iloc[:, 1:].head()

# add a prefix to identify the source of the dummy variables
pd.get_dummies(train.Sex, prefix='Sex').iloc[:, 1:].head()

# use 'get_dummies' with a feature that has 3 possible values
pd.get_dummies(train.Embarked, prefix='Embarked').head(10)


# drop the first dummy variable ('C')
pd.get_dummies(train.Embarked, prefix='Embarked').iloc[:, 1:].head(10)

# save the DataFrame of dummy variables and concatenate them to the original DataFrame
embarked_dummies = pd.get_dummies(train.Embarked, prefix='Embarked').iloc[:, 1:]
train = pd.concat([train, embarked_dummies], axis=1)
train.head()

# reset the DataFrame
train = pd.read_csv('http://bit.ly/kaggletrain')
train.head()


# pass the DataFrame to 'get_dummies' and specify which columns to dummy (it drops the original columns)
pd.get_dummies(train, columns=['Sex', 'Embarked']).head()
                                            
                                            
pd.get_dummies(train,columns=['Sex','Embarked'],drop_first=True)

### split_list code

In [60]:
### Create dataframe containing split string values by respondent number
# more efficient than expanding along bool columns for each language
def split_list(df, col):
    """Create a new dataframe that splits the values of multi-selection column col into individual selections and 
    places each selection value on a separate row. This new dataframe can be linked back to the original dataframe by 
    Respondent value.
    
    Args:
    df: dataframe. Dataframe containing the multi-selection field col.
       
    Returns:
    out_df: dataframe. New dataframe giving split values of col.
    """
    
    in_res = list(df.index)
    in_list = list(df[col])
    
    out_res = []
    out_list = []
    
    for i in range(len(in_list)):
        if pd.isnull(in_list[i]) == False:
            vals = in_list[i].split(';')
            res = [in_res[i]]*len(vals)
            
            out_list.append(vals)
            out_res.append(res)
    
    out_df = pd.DataFrame({'Respondent': list(np.concatenate(out_res)), col: list(np.concatenate(out_list))})
    
    return out_df

split_list(survey,'LanguageWorkedWith')
#survey.index

,Respondent,LanguageWorkedWith
0,4,C
1,4,C++
2,4,C#
3,4,Python
4,4,SQL
5,5,C++
6,5,HTML/CSS
7,5,Java
8,5,JavaScript
9,5,Python
